In [1]:
import os
import s3fs
import dask
import dash
import requests

from dash import dcc, html
from dash.dependencies import Input, Output
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import xarray as xr
import pandas as pd 
import cartopy.crs as ccrs
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.dates as mdates 
import cartopy.feature as cfeature
import matplotlib.patheffects as path_effects


In [2]:
# load in methods for reading, writing, and managing files stored in S3. The connection is made anonamously. 
s3 = s3fs.S3FileSystem(anon=True) 
# List all data folders available from UNSW
s3_bucket_name = "imos-data/"
s3_org_name = "UNSW/"
s3_product = "NRS_extremes/Temperature_DataProducts_v2"
data_folders = s3.ls(f"{s3_bucket_name}{s3_org_name}{s3_product}")
# List all subfolders under the product directory
data_folders

['imos-data/UNSW/NRS_extremes/Temperature_DataProducts_v2/MAI090',
 'imos-data/UNSW/NRS_extremes/Temperature_DataProducts_v2/PH050',
 'imos-data/UNSW/NRS_extremes/Temperature_DataProducts_v2/PH100',
 'imos-data/UNSW/NRS_extremes/Temperature_DataProducts_v2/ROT055']

## Load datasets into Memory (explicit load)

In [3]:
# Initialize an empty dictionary to store datasets
datasets = {}

# Iterate through each subfolder and load .nc files
for folder in data_folders:
    # List files in the current folder
    files_in_folder = s3.ls(folder)
    
    # Filter for .nc files
    nc_files = [file for file in files_in_folder if file.endswith(".nc")]
    
    # Load each .nc file into memory
    for nc_file in nc_files:
        # Open the file using s3fs and load the dataset into memory with xarray
        with s3.open(nc_file, mode='rb') as f:
            ds = xr.open_dataset(f, engine="h5netcdf")
            ds.load()  # Explicitly load the dataset into memory
        
        # Add to the datasets dictionary using the folder name as the key
        datasets[folder] = ds

In [5]:
datasets

{'imos-data/UNSW/NRS_extremes/Temperature_DataProducts_v2/MAI090': <xarray.Dataset> Size: 8MB
 Dimensions:                                 (TIME: 29176, DEPTH: 2)
 Coordinates:
   * TIME                                    (TIME) datetime64[ns] 233kB 1944-...
   * DEPTH                                   (DEPTH) float32 8B 2.0 21.0
 Data variables: (12/35)
     TEMP                                    (TIME, DEPTH) float32 233kB nan ....
     TEMP_INTERP                             (TIME, DEPTH) float32 233kB nan ....
     TEMP_HEAT_SPIKE                         (TIME, DEPTH) float32 233kB nan ....
     TEMP_COLD_SPIKE                         (TIME, DEPTH) float32 233kB nan ....
     TEMP_EXTREME_INDEX                      (TIME, DEPTH) float32 233kB nan ....
     TEMP_PER10                              (TIME, DEPTH) float32 233kB 14.37...
     ...                                      ...
     MCS_EVENT_INTENSITY_VAR                 (TIME, DEPTH) float32 233kB nan ....
     MCS_EVENT_INTE

##  Show location of stations in the Climatology directory

In [ ]:
# Create a figure and a map with coastlines
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})

# Add coastlines and land features
ax.coastlines(resolution='50m')
ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')

# Set the extent to focus on Australia (bounding box coordinates)
ax.set_extent([110, 160, -45, -10], crs=ccrs.PlateCarree())  # [lon_min, lon_max, lat_min, lat_max]


# Iterate over datasets to plot the "O" at geospatial_lat_max and geospatial_lon_max
for folder, ds in datasets.items():
    # Extract latitude and longitude from the dataset attributes
    lat = ds.attrs.get('geospatial_lat_max')
    lon = ds.attrs.get('geospatial_lon_max')
    
    # Check if the attributes exist before plotting
    if lat is not None and lon is not None:
        # Plot a big "O" at the location
        ax.text(lon, lat, 'O', fontsize=10, color='red', ha='center', va='center',
                transform=ccrs.PlateCarree())

# Set title and show plot
ax.set_title('Locations of Geospatial Maxima')
plt.show()

## Create and interactive date picker with the colour coded spikes

In [ ]:
# Function to convert date to "days since 1950-01-01"
def date_to_days_since_1950(date):
    ref_date = pd.to_datetime("1950-01-01")
    delta = date - ref_date
    return delta.days

# Function to update the plot
def update_plot(date):
    # Convert selected date to "days since 1950-01-01"
    target_days = date_to_days_since_1950(pd.to_datetime(date))
    
    # Create a figure and a map with coastlines
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')
    ax.set_extent([110, 160, -45, -10], crs=ccrs.PlateCarree())

    # Iterate over datasets to plot markers
    for folder, ds in datasets.items():
        lat = ds.attrs.get('geospatial_lat_max')
        lon = ds.attrs.get('geospatial_lon_max')
        
        if lat is not None and lon is not None:
            # TIME is already in datetime64 format, so no need for timedelta conversion
            time_in_days = (ds['TIME'].values - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')

            # Find the index for the selected date in days since 1950
            date_idx = np.isclose(time_in_days, target_days, atol=0.5)  # use a tolerance for floating point comparisons
            
            if date_idx.any():
                temp_heat_spike = ds['TEMP_HEAT_SPIKE'].values[date_idx]
                temp_cold_spike = ds['TEMP_COLD_SPIKE'].values[date_idx]
                
                if not pd.isna(temp_heat_spike).all():
                    ax.plot(lon, lat, 'o', markersize=10, color='red', markeredgecolor='black', markeredgewidth=1.5,
                            transform=ccrs.PlateCarree())
                elif not pd.isna(temp_cold_spike).all():
                    ax.plot(lon, lat, 'o', markersize=10, color='blue', markeredgecolor='black', markeredgewidth=1.5,
                            transform=ccrs.PlateCarree())
                else:
                    ax.text(lon, lat, 'O', fontsize=15, color='black', ha='center', va='center',
                            transform=ccrs.PlateCarree())
    
    ax.set_title(f'Heat and Cold Spikes on {date}')
    plt.show()

# Create a date picker widget
date_picker = widgets.DatePicker(
    description='Select Date',
    value=pd.to_datetime("2018-11-30") # Date happens to be a Heatwave and a Cold spike at the same time east and west.
)

# Create an interactive function to trigger the plot update when the date changes
widgets.interactive(update_plot, date=date_picker)



## Attempt at Bokeh to HTML

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, curdoc
from bokeh.models import HoverTool, WMTSTileSource, ColumnDataSource
from bokeh.layouts import column
from bokeh.models.widgets import DatePicker
import numpy as np
import pandas as pd

output_notebook()

# Sample dataset for illustration
datasets = {
    "dataset_1": {"attrs": {"geospatial_lat_max": -33.8688, "geospatial_lon_max": 151.2093},
                  "TIME": np.array(['2018-11-30T00:00:00'], dtype='datetime64[ns]'),
                  "TEMP_HEAT_SPIKE": np.array([1]), "TEMP_COLD_SPIKE": np.array([np.nan])},
    "dataset_2": {"attrs": {"geospatial_lat_max": -31.9505, "geospatial_lon_max": 115.8605},
                  "TIME": np.array(['2018-11-30T00:00:00'], dtype='datetime64[ns]'),
                  "TEMP_HEAT_SPIKE": np.array([np.nan]), "TEMP_COLD_SPIKE": np.array([1])}
}

# Function to convert lat/lon to Mercator projection
def lon_lat_to_mercator(lon, lat):
    k = 6378137
    x = lon * (k * np.pi / 180.0)
    y = np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    return x, y

# Function to convert date to "days since 1950-01-01"
def date_to_days_since_1950(date):
    ref_date = pd.to_datetime("1950-01-01")
    delta = date - ref_date
    return delta.days

# Set up Bokeh figure with custom tile provider
p = figure(x_range=(11000000, 16000000), y_range=(-4500000, -1000000), 
           height=800, width=800,
           x_axis_type="mercator", y_axis_type="mercator")

tile_url = "https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"
p.add_tile(WMTSTileSource(url=tile_url))

# Set up data source for interactive updating
source = ColumnDataSource(data=dict(x=[], y=[], color=[], size=[]))

p.circle(x='x', y='y', color='color', size='size', source=source, line_color='black', line_width=1.5)

hover = HoverTool(tooltips=[("Lat", "@y"), ("Lon", "@x")])
p.add_tools(hover)

# Function to update the plot based on the selected date
def update_plot_bokeh_interactive(attr, old, new):
    selected_date = pd.to_datetime(date_picker.value)
    target_days = date_to_days_since_1950(selected_date)
    
    # Clear current data
    new_data = dict(x=[], y=[], color=[], size=[])
    
    # Plot markers based on dataset
    for folder, ds in datasets.items():
        lat = ds['attrs']['geospatial_lat_max']
        lon = ds['attrs']['geospatial_lon_max']
        
        time_in_days = (ds['TIME'] - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')
        
        date_idx = np.isclose(time_in_days, target_days, atol=0.5)
        
        if date_idx.any():
            temp_heat_spike = ds['TEMP_HEAT_SPIKE'][date_idx]
            temp_cold_spike = ds['TEMP_COLD_SPIKE'][date_idx]
            
            lon_merc, lat_merc = lon_lat_to_mercator(lon, lat)
            
            if not pd.isna(temp_heat_spike).all():
                new_data['x'].append(lon_merc)
                new_data['y'].append(lat_merc)
                new_data['color'].append("red")
                new_data['size'].append(10)
            elif not pd.isna(temp_cold_spike).all():
                new_data['x'].append(lon_merc)
                new_data['y'].append(lat_merc)
                new_data['color'].append("blue")
                new_data['size'].append(10)
    
    # Update data source
    source.data = new_data
    p.title.text = f'Heat and Cold Spikes on {selected_date.date()}'

# Create a DatePicker widget
date_picker = DatePicker(title="Select Date", value="2018-11-30", min_date="2010-01-01", max_date="2022-12-31")
date_picker.on_change('value', update_plot_bokeh_interactive)

# Initial update for the plot
update_plot_bokeh_interactive(None, None, None)

# Layout and display
layout = column(date_picker, p)
curdoc().add_root(layout)
show(layout)


## Attempt at plotly plot

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets

# Initialize a Plotly FigureWidget object
fig = go.FigureWidget()

# Function to convert date to "days since 1950-01-01"
def date_to_days_since_1950(date):
    ref_date = pd.to_datetime("1950-01-01")
    delta = date - ref_date
    return delta.days

# Function to update the plot
def update_plot(date):
    # Convert selected date to "days since 1950-01-01"
    target_days = date_to_days_since_1950(pd.to_datetime(date))
    
    # Prepare lists to hold marker data
    lons = []
    lats = []
    colors = []
    text = []
    symbols = []

    # Iterate over datasets to gather marker data
    for folder, ds in datasets.items():
        lat = ds.attrs.get('geospatial_lat_max')
        lon = ds.attrs.get('geospatial_lon_max')
        
        if lat is not None and lon is not None:
            time_in_days = (ds['TIME'].values - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')
            date_idx = np.isclose(time_in_days, target_days, atol=0.5)
            
            if date_idx.any():
                temp_heat_spike = ds['TEMP_HEAT_SPIKE'].values[date_idx]
                temp_cold_spike = ds['TEMP_COLD_SPIKE'].values[date_idx]
                
                if not pd.isna(temp_heat_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('red')
                    text.append('Heat Spike')
                    symbols.append('circle')  # Solid circle for heat spikes
                elif not pd.isna(temp_cold_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('blue')
                    text.append('Cold Spike')
                    symbols.append('circle')  # Solid circle for cold spikes
                else:
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('black')
                    text.append('No Data')
                    symbols.append('circle-open')  # Hollow circle for no data

    # Update the traces in the FigureWidget
    with fig.batch_update():
        fig.data = []  # Clear existing traces
        
        fig.add_trace(go.Scattergeo(
            lon=lons,
            lat=lats,
            mode='markers',
            marker=dict(
                size=10,
                color=colors,
                symbol=symbols,  # Use symbols to set marker shapes
                line=dict(
                    width=2,
                    color='black'
                ),
                opacity=0.6
            ),
            text=text,
            hoverinfo='text'
        ))
        
        fig.update_layout(
            title=f'Heat and Cold Spikes on {date}',
            geo=dict(
                scope='world',
                projection_type='mercator',
                center=dict(lat=-27.0, lon=135.0),  # Center around Australia
                projection=dict(
                    type='mercator'
                ),
                showland=True,
                landcolor='lightgrey',
                coastlinecolor='black',
                showocean=True,
                oceancolor='lightblue',
                showlakes=True,
                lakecolor='lightblue',
                showrivers=True,
                lonaxis=dict(
                    range=[100, 160]  # Adjust longitude bounds to cover all of Australia
                ),
                lataxis=dict(
                    range=[-50, -10]  # Adjust latitude bounds to cover all of Australia
                )
            ),
            autosize=True,
            height=800,  # Increase the height of the figure
            width=1000   # Increase the width of the figure
        )

# Create a date picker widget
date_picker = widgets.DatePicker(
    description='Select Date',
    value=pd.to_datetime("2018-11-30")
)

# Create an interactive function to trigger the plot update when the date changes
interactive_plot = widgets.interactive(update_plot, date=date_picker)

# Display the date picker and plot together
display(date_picker, fig)


## Attempt to create a time slider instead of drop down

In [ ]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Function to convert date to "days since 1950-01-01"
def date_to_days_since_1950(date):
    ref_date = pd.to_datetime("1950-01-01")
    delta = date - ref_date
    return delta.days

# Function to convert "days since 1950-01-01" to a date
def days_since_1950_to_date(days):
    ref_date = pd.to_datetime("1950-01-01")
    return ref_date + pd.to_timedelta(days, unit='D')

# Function to update the plot
def update_plot(days):
    # Convert selected days to a date
    date = days_since_1950_to_date(days)
    
    # Create a figure and a map with coastlines
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')
    ax.set_extent([110, 160, -45, -10], crs=ccrs.PlateCarree())

    # Iterate over datasets to plot markers
    for folder, ds in datasets.items():
        lat = ds.attrs.get('geospatial_lat_max')
        lon = ds.attrs.get('geospatial_lon_max')
        
        if lat is not None and lon is not None:
            # TIME is already in datetime64 format, so no need for timedelta conversion
            time_in_days = (ds['TIME'].values - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')

            # Find the index for the selected date in days since 1950
            date_idx = np.isclose(time_in_days, days, atol=0.5)  # use a tolerance for floating point comparisons
            
            if date_idx.any():
                temp_heat_spike = ds['TEMP_HEAT_SPIKE'].values[date_idx]
                temp_cold_spike = ds['TEMP_COLD_SPIKE'].values[date_idx]
                
                if not pd.isna(temp_heat_spike).all():
                    ax.plot(lon, lat, 'o', markersize=10, color='red', markeredgecolor='black', markeredgewidth=1.5,
                            transform=ccrs.PlateCarree())
                elif not pd.isna(temp_cold_spike).all():
                    ax.plot(lon, lat, 'o', markersize=10, color='blue', markeredgecolor='black', markeredgewidth=1.5,
                            transform=ccrs.PlateCarree())
                else:
                    ax.text(lon, lat, 'O', fontsize=15, color='black', ha='center', va='center',
                            transform=ccrs.PlateCarree())
    
    ax.set_title(f'Heat and Cold Spikes on {date.date()}')
    plt.show()

# Determine the maximum number of days since 1950 based on the most recent dataset's date
latest_date = pd.to_datetime("today")  # Replace with the latest date in your data, or use today's date
max_days = date_to_days_since_1950(latest_date)

# Create a slider with the correct range
date_slider = widgets.IntSlider(
    value=date_to_days_since_1950(pd.to_datetime("2018-11-30")),  # Initial value
    min=0,  # Start from 1950-01-01
    max=max_days,  # Maximum number of days until today or your latest data
    step=1,  # Step in days
    description='Days since 1950',
    layout=widgets.Layout(width='800px')  # Adjust width of the slider
)

# Create an interactive function to trigger the plot update when the slider changes
interactive_plot = widgets.interactive(update_plot, days=date_slider)

# Display the slider and plot without duplicating the slider
widgets.VBox([interactive_plot])


## Atempt to use Plotly with HTML output

In [ ]:
# Convert TIME variable to actual dates
def convert_days_to_dates(time_in_days):
    base_date = pd.Timestamp("1950-01-01")
    return base_date + pd.to_timedelta(time_in_days, unit='D')

for folder, ds in datasets.items():
    time_in_days = ds['TIME'].values
    if np.issubdtype(time_in_days.dtype, np.datetime64):
        time_in_days = (time_in_days - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')
    ds['dates'] = convert_days_to_dates(time_in_days)

# Extract unique years, months, and days for dropdowns
all_dates = pd.to_datetime(np.concatenate([ds['dates'].values for ds in datasets.values()]))
years = sorted(all_dates.year.unique())
months = sorted(all_dates.month.unique())
days = {month: sorted(all_dates[all_dates.month == month].day.unique()) for month in months}

# Create a list of all available dates
available_dates = sorted(set(pd.to_datetime(np.concatenate([ds['dates'].values for ds in datasets.values()]))))

# Create dropdown buttons
def create_buttons(dates):
    buttons = []
    for date in dates:
        buttons.append(dict(
            label=date.strftime('%Y-%m-%d'),
            method='update',
            args=[{'title': f'Heat and Cold Spikes on {date.date()}'}]
        ))
    return buttons

# Function to create the plot based on selected date
def create_figure(selected_date):
    lons = []
    lats = []
    colors = []
    text = []

    for folder, ds in datasets.items():
        date_mask = ds['dates'].values == selected_date

        if date_mask.any():
            lat = ds.attrs.get('geospatial_lat_max')
            lon = ds.attrs.get('geospatial_lon_max')

            if lat is not None and lon is not None:
                temp_heat_spike = ds['TEMP_HEAT_SPIKE'].values[date_mask]
                temp_cold_spike = ds['TEMP_COLD_SPIKE'].values[date_mask]

                if not np.isnan(temp_heat_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('red')
                    text.append('Heat Spike')
                elif not np.isnan(temp_cold_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('blue')
                    text.append('Cold Spike')
                else:
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('black')
                    text.append('No Data')

    fig = go.Figure(go.Scattergeo(
        lon=lons,
        lat=lats,
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            line=dict(
                width=2,
                color='black'
            ),
            opacity=0.6
        ),
        text=text,
        hoverinfo='text'
    ))

    fig.update_layout(
        title=f'Heat and Cold Spikes on {selected_date.date()}',
        geo=dict(
            scope='world',
            projection_type='mercator',
            center=dict(lat=-27.0, lon=135.0),
            showland=True,
            landcolor='lightgrey',
            coastlinecolor='black',
            showocean=True,
            oceancolor='lightblue',
            showlakes=True,
            lakecolor='lightblue',
            showrivers=True,
            lonaxis=dict(range=[100, 160]),
            lataxis=dict(range=[-50, -10])
        ),
        height=800,
        width=1000,
        updatemenus=[{
            'buttons': create_buttons(available_dates),
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }]
    )

    return fig

# Generate the initial plot for a specific date
selected_date = available_dates[0]  # Use the first available date as default
fig = create_figure(selected_date)
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Initialize sample datasets
datasets = {
    'folder1': {'attrs': {'geospatial_lat_max': -25.0, 'geospatial_lon_max': 135.0},
                 'TIME': np.array(pd.date_range(start='1950-01-01', periods=100, freq='D')),
                 'TEMP_HEAT_SPIKE': np.random.rand(100),
                 'TEMP_COLD_SPIKE': np.random.rand(100)},
}

def date_to_days_since_1950(date):
    ref_date = pd.to_datetime("1950-01-01")
    delta = date - ref_date
    return delta.days

def generate_fig(date):
    target_days = date_to_days_since_1950(pd.to_datetime(date))
    
    lons = []
    lats = []
    colors = []
    text = []
    symbols = []

    for folder, ds in datasets.items():
        lat = ds['attrs'].get('geospatial_lat_max')
        lon = ds['attrs'].get('geospatial_lon_max')

        if lat is not None and lon is not None:
            time_in_days = (ds['TIME'] - np.datetime64("1950-01-01")) / np.timedelta64(1, 'D')
            date_idx = np.isclose(time_in_days, target_days, atol=0.5)

            if date_idx.any():
                temp_heat_spike = ds['TEMP_HEAT_SPIKE'][date_idx]
                temp_cold_spike = ds['TEMP_COLD_SPIKE'][date_idx]

                if not np.isnan(temp_heat_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('red')
                    text.append('Heat Spike')
                    symbols.append('circle')
                elif not np.isnan(temp_cold_spike).all():
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('blue')
                    text.append('Cold Spike')
                    symbols.append('circle')
                else:
                    lons.append(lon)
                    lats.append(lat)
                    colors.append('black')
                    text.append('No Data')
                    symbols.append('circle-open')

    fig = go.Figure()
    fig.add_trace(go.Scattergeo(
        lon=lons,
        lat=lats,
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            symbol=symbols,
            line=dict(width=2, color='black'),
            opacity=0.6
        ),
        text=text,
        hoverinfo='text'
    ))

    fig.update_layout(
        title=f'Heat and Cold Spikes on {date}',
        geo=dict(
            scope='world',
            projection_type='mercator',
            center=dict(lat=-27.0, lon=135.0),
            lonaxis=dict(range=[100, 160]),
            lataxis=dict(range=[-50, -10])
        ),
        autosize=True,
        height=800,
        width=1000
    )
    return fig

# Example usage
fig = generate_fig('2018-11-30')

# Add a slider to the figure
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': True,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ],
    sliders=[{
        'active': 0,
        'steps': [{
            'label': '2018-11-30',
            'method': 'update',
            'args': [{'visible': [True]}]
        }]
    }]
)

fig.write_html('interactive_plot_with_slider.html')
